# Notebook 1: DB Health Check

Quick checks to see if the database looks OK.

## Connect & Load

Connect to the database and load tables.

In [1]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timezone

ROOT_DIR = Path().resolve().parents[0]
sys.path.append(str(ROOT_DIR))

from src.utils.db_utils import (
    load_table,
    load_all_tables,
    DB_HOST,
    DB_PORT,
    DB_NAME,
 )

def env_stamp():
    print("--- Environment ---")
    print("Timestamp:", datetime.now(timezone.utc).isoformat())
    print("DB:", DB_HOST, DB_PORT, DB_NAME)

def safe_load(table_name):
    try:
        df = load_table(table_name)
        print(f"{'Table:':<8} {table_name:<28} | Rows: {df.shape[0]:>6} | Columns: {df.shape[1]:>6}")
        return df
    except Exception as e:
        print(f"{'Table:':<8} {table_name:<28} | Failed to load | Error: {e}")
        return pd.DataFrame()

def profile_tables(tables_dict):
    print("--- Table Profile ---")
    for name, df in tables_dict.items():
        print(f"{name:<25} | Rows: {len(df):>8} | Cols: {df.shape[1]:>4}")

def infer_key_columns(tables_dict):
    key_map = {}
    for name, df in tables_dict.items():
        cols = []
        for c in df.columns:
            if c.endswith("_key") or c == "id" or c == "year":
                cols.append(c)
        if cols:
            key_map[name] = sorted(set(cols))
    return key_map

def null_checks(tables_dict, key_columns):
    print("--- Null Checks ---")
    for name, df in tables_dict.items():
        cols = key_columns.get(name, [])
        for c in cols:
            if c in df.columns:
                nulls = df[c].isna().sum()
                print(f"{name:<25} | {c:<20} | Nulls: {nulls}")

def range_checks(df, checks):
    print("--- Range Checks ---")
    for col, (low, high) in checks.items():
        if col in df.columns:
            series = df[col].dropna()
            too_low = (series < low).sum()
            too_high = (series > high).sum() if high is not None else 0
            print(f"{col:<25} | <{low}: {too_low} | >{high}: {too_high}")

def fk_check(fact_df, fact_key, dim_df, dim_key, label):
    if fact_key not in fact_df.columns or dim_key not in dim_df.columns:
        print(f"{label:<60} | Skipped (missing columns)")
        return
    fact_keys = set(fact_df[fact_key].dropna().unique())
    dim_keys = set(dim_df[dim_key].dropna().unique())
    missing = sorted(fact_keys - dim_keys)
    examples = missing[:5]
    print(f"{label:<60} | Missing: {len(missing):<6} | Missing IDs: {examples}")

def infer_fk_pairs(tables_dict):
    dim_tables = {name: df for name, df in tables_dict.items() if name.lower().startswith("dim")}
    fk_pairs = []

    for fact_name, fact_df in tables_dict.items():
        if not fact_name.lower().startswith("fact"):
            continue
        for col in fact_df.columns:
            if not col.endswith("_key"):
                continue
            base = col.replace("_key", "")
            dim_name = "Dim" + base[:1].upper() + base[1:]
            if dim_name in dim_tables:
                fk_pairs.append((fact_name, col, dim_name, "id"))
    return fk_pairs

env_stamp()
tables = load_all_tables(denylist=["alembic_version"])
print()
profile_tables(tables)

--- Environment ---
Timestamp: 2026-02-16T17:35:57.289484+00:00
DB: localhost 5432 global_impact

--- Table Profile ---
DimCountry                | Rows:      211 | Cols:    8
DimRegion                 | Rows:        7 | Cols:    3
DimScenario               | Rows:        2 | Cols:    3
DimSector                 | Rows:       11 | Cols:    2
DimYear                   | Rows:      100 | Cols:    3
FactCountryYearMetrics    | Rows:    13504 | Cols:   14
FactSectorEmissions       | Rows:    68959 | Cols:    6


In [2]:
for name, df in tables.items():
    print(f"--- {name} ---")
    
    if name.lower().startswith("fact"):
        display(df.iloc[np.r_[0:5, -5:0]])
    else:
        display(df.head())
    
    print()


--- DimCountry ---


,id,country_name,iso2,iso3,region_key,capital,longitude,latitude
0,1,Afghanistan,AF,AFG,4,Kabul,69.17610,34.5228
1,2,Albania,AL,ALB,2,Tirane,19.81720,41.3317
2,3,Algeria,DZ,DZA,4,Algiers,3.05097,36.7397
3,4,American Samoa,AS,ASM,1,Pago Pago,-170.69100,-14.2846
4,5,Andorra,AD,AND,2,Andorra la Vella,1.52180,42.5075



--- DimRegion ---


,id,region_id,region_name
0,1,EAS,East Asia & Pacific
1,2,ECS,Europe & Central Asia
2,3,LCN,Latin America & Caribbean
3,4,MEA,"Middle East, North Africa, Afghanistan & Pakistan"
4,5,NAC,North America



--- DimScenario ---


...

,id,year,decade
0,1,1950,1950
1,2,1951,1950
2,3,1952,1950
3,4,1953,1950
4,5,1954,1950



--- FactCountryYearMetrics ---


,id,country_key,year_key,scenario_key,population,total_co2_emissions,methane_emissions,total_ghg_emissions,energy_consumption_total,electricity_consumption,renewable_energy_share,gdp,data_source,last_updated
0,1,1,11,1,9035043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World Bank,2026-01-28
1,2,1,12,1,9214083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World Bank,2026-01-28
2,3,1,13,1,9404406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World Bank,2026-01-28
3,4,1,14,1,9604487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World Bank,2026-01-28
4,5,1,15,1,9814318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World Bank,2026-01-28
13499,13500,211,70,1,15526888,9.0746,11.2028,24.7938,373.557246,464.806599,84.1,3.198033e+10,World Bank,2026-01-28
13500,13501,211,71,1,15797210,10.8831,11.5358,27.1497,405.518564,540.095371,82.4,4.128767e+10,World Bank,2026-01-28
13501,13502,211,72,1,16069056,11.5572,11.7157,28.0256,416.586875,538.799541,NaN,4.075756e+10,World Bank,2026-01-28
13502,13503,211,73,1,16340822,12.5660,12.5031,30.1250,NaN,NaN,NaN,3.587178e+10,World Bank,2026-01-28
13503,13504,211,74,1,16634373,12.8554,12.7896,30.7843,NaN,NaN,NaN,4.153941e+10,World Bank,2026-01-28



--- FactSectorEmissions ---


,id,year_key,country_key,sector_key,scenario_key,emissions
0,1,41,1,1,1,8410000.0
1,2,42,1,1,1,8750000.0
2,3,43,1,1,1,8780000.0
3,4,44,1,1,1,8860000.0
4,5,45,1,1,1,8940000.0
68954,68955,69,211,11,1,170000.0
68955,68956,70,211,11,1,70000.0
68956,68957,71,211,11,1,129999.0
68957,68958,72,211,11,1,190000.0
68958,68959,73,211,11,1,180000.0


## Data Shape & Integrity

Check columns, types, and duplicates.

In [3]:
for name, df in tables.items():
    print(f"--- {name} ---")
    display(df.dtypes)
    print()


--- DimCountry ---


id                int64
country_name        str
iso2                str
iso3                str
region_key        int64
capital             str
longitude       float64
latitude        float64
dtype: object


--- DimRegion ---


id             int64
region_id        str
region_name      str
dtype: object


--- DimScenario ---


...

id        int64
year      int64
decade    int64
dtype: object


--- FactCountryYearMetrics ---


id                                   int64
country_key                          int64
year_key                             int64
scenario_key                         int64
population                           int64
total_co2_emissions                float64
methane_emissions                  float64
total_ghg_emissions                float64
energy_consumption_total           float64
electricity_consumption            float64
renewable_energy_share             float64
gdp                                float64
data_source                            str
last_updated                datetime64[us]
dtype: object


--- FactSectorEmissions ---


id                int64
year_key          int64
country_key       int64
sector_key        int64
scenario_key      int64
emissions       float64
dtype: object

In [4]:
print("--- Checking for duplicates ---")

for name, df in tables.items():
    dup_count = df.duplicated().sum()
    print(f"{name:<30} | Duplicates: {dup_count}")


--- Checking for duplicates ---
DimCountry                     | Duplicates: 0
DimRegion                      | Duplicates: 0
DimScenario                    | Duplicates: 0
DimSector                      | Duplicates: 0
DimYear                        | Duplicates: 0
FactCountryYearMetrics         | Duplicates: 0
FactSectorEmissions            | Duplicates: 0


## Health Checks

Null checks.

In [5]:
key_columns = infer_key_columns(tables)
null_checks(tables, key_columns)

--- Null Checks ---
DimCountry                | id                   | Nulls: 0
DimCountry                | region_key           | Nulls: 0
DimRegion                 | id                   | Nulls: 0
DimScenario               | id                   | Nulls: 0
DimSector                 | id                   | Nulls: 0
DimYear                   | id                   | Nulls: 0
DimYear                   | year                 | Nulls: 0
FactCountryYearMetrics    | country_key          | Nulls: 0
FactCountryYearMetrics    | id                   | Nulls: 0
FactCountryYearMetrics    | scenario_key         | Nulls: 0
FactCountryYearMetrics    | year_key             | Nulls: 0
FactSectorEmissions       | country_key          | Nulls: 0
FactSectorEmissions       | id                   | Nulls: 0
FactSectorEmissions       | scenario_key         | Nulls: 0
FactSectorEmissions       | sector_key           | Nulls: 0
FactSectorEmissions       | year_key             | Nulls: 0


## Cross-table Relationships

Check if fact keys exist in dimension tables.

In [6]:
print("--- Cross-table Relationships (Auto) ---")
for fact_name, fact_key, dim_name, dim_key in infer_fk_pairs(tables):
    fk_check(tables[fact_name], fact_key, tables[dim_name], dim_key,
             f"{fact_name}.{fact_key} -> {dim_name}.{dim_key}")

--- Cross-table Relationships (Auto) ---
FactCountryYearMetrics.country_key -> DimCountry.id          | Missing: 0      | Missing IDs: []
FactCountryYearMetrics.year_key -> DimYear.id                | Missing: 0      | Missing IDs: []
FactCountryYearMetrics.scenario_key -> DimScenario.id        | Missing: 0      | Missing IDs: []
FactSectorEmissions.year_key -> DimYear.id                   | Missing: 0      | Missing IDs: []
FactSectorEmissions.country_key -> DimCountry.id             | Missing: 0      | Missing IDs: []
FactSectorEmissions.sector_key -> DimSector.id               | Missing: 0      | Missing IDs: []
FactSectorEmissions.scenario_key -> DimScenario.id           | Missing: 0      | Missing IDs: []


## Basic Aggregations

Quick totals by year.

In [7]:
print("--- Basic Aggregations ---")

df_metrics = tables.get("FactCountryYearMetrics", pd.DataFrame())
df_dimYear = tables.get("DimYear", pd.DataFrame())

if df_metrics.empty or df_dimYear.empty:
    print("Skipping aggregations (missing FactCountryYearMetrics or DimYear).")
else:
    df_metrics_year = df_metrics.merge(
        df_dimYear[["id", "year"]], left_on="year_key", right_on="id", how="left"
    ).rename(columns={"year": "year_value"})

    agg = (
        df_metrics_year.groupby("year_value", dropna=True)
        .agg(
            population=("population", "sum"),
            total_co2=("total_co2_emissions", "sum"),
            total_ghg=("total_ghg_emissions", "sum"),
            energy=("energy_consumption_total", "sum"),
            electricity=("electricity_consumption", "sum"),
            renewable_share=("renewable_energy_share", "mean"),
            gdp=("gdp", "sum"),
        )
        .reset_index()
        .sort_values("year_value")
    )

    display(agg.tail(10))

--- Basic Aggregations ---


,year_value,population,total_co2,total_ghg,energy,electricity,renewable_share,gdp
54,2015,7397434864,34614.9119,46666.4249,351680.707465,598173.949259,29.696135,7.438429e+13
55,2016,7484271894,34663.3979,46819.1510,348659.094480,599000.498279,29.444928,7.555177e+13
56,2017,7569562671,35231.8843,47580.0215,350971.649215,603961.793962,29.274396,8.044652e+13
57,2018,7651896272,36101.6270,48648.8750,350444.764043,609905.792857,29.319324,8.559436e+13
58,2019,7732297251,36221.1795,48928.5715,345586.350970,609391.773738,29.131884,8.697445e+13
59,2020,7808777424,34824.3282,47469.5947,326927.749017,595662.272975,30.117874,8.438197e+13
60,2021,7874433997,36772.6747,49633.0458,342693.022337,619582.871968,29.610628,9.627249e+13
61,2022,7943529603,36963.0476,50052.4983,338027.348792,612495.706620,30.044928,1.003629e+14
62,2023,8017248810,37386.1083,50700.5404,183628.334770,407801.498947,NaN,1.047846e+14
63,2024,8094713939,37881.7031,51341.2332,0.000000,0.000000,NaN,1.088512e+14
